In [ ]:
from flask import Flask, render_template, request
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed, Attention, Concatenate, Input
import re

# Initialize the Flask application
app = Flask(__name__)

# Load the pre-trained translation model, including any custom layers used during training
model_path = "translation_model.h5" #Enter the path of your pre-trained model
model = load_model(model_path, custom_objects={
    'Bidirectional': Bidirectional,
    'Attention': Attention,
    'Concatenate': Concatenate
})

# Initialize tokenizers for English and French languages (to be consistent with the ones used during model training)
tokenizer_english = Tokenizer(oov_token='<UNK>')
tokenizer_french = Tokenizer(oov_token='<UNK>')

# Example word indices (Replace with the actual indices from your trained model)
# These should match the tokenizers used during the training process
tokenizer_english.word_index = {'<UNK>': 1, 'hello': 2, 'world': 3}  # Replace with the actual word index for English
tokenizer_french.word_index = {'<UNK>': 1, 'bonjour': 2, 'monde': 3}  # Replace with the actual word index for French

def preprocess_input(text, tokenizer, max_len):
    """
    Preprocess the input text by converting it to a sequence of tokens and padding the sequence.
    
    Args:
        text (str): The input sentence to translate.
        tokenizer (Tokenizer): The tokenizer corresponding to the input language.
        max_len (int): The maximum sequence length.
    
    Returns:
        np.array: The padded sequence of tokens.
    """
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
    return padded_sequences

def decode_output(predictions, tokenizer):
    """
    Decode the model's output to a human-readable sentence.
    
    Args:
        predictions (np.array): The predicted sequence of token indices.
        tokenizer (Tokenizer): The tokenizer corresponding to the output language.
    
    Returns:
        str: The translated sentence.
    """
    index_word = {index: word for word, index in tokenizer.word_index.items()}
    decoded_sentence = ' '.join([index_word.get(index, '') for index in predictions[0] if index != 0])
    return decoded_sentence

@app.route('/')
def index():
    """
    Render the main page of the translation web app.
    
    Returns:
        str: The rendered HTML of the index page.
    """
    return render_template('index.html')

@app.route('/translate', methods=['POST'])
def translate():
    """
    Handle the translation request from the user.
    
    Returns:
        str: The translated sentence.
    """
    sentence = request.form['sentence']  # Get the input sentence from the form
    max_len = 30  # The maximum sequence length used during model training
    
    # Preprocess the input sentence
    input_sequence = preprocess_input(sentence, tokenizer_english, max_len)
    
    # Prepare a dummy decoder input (typically this would be more complex)
    decoder_input = np.zeros((1, max_len))

    # Get model predictions
    predictions = model.predict([input_sequence, decoder_input])

    # Decode the predictions to a human-readable sentence
    translated_sequence = np.argmax(predictions, axis=-1)
    translated_sentence = decode_output(translated_sequence, tokenizer_french)
    
    return translated_sentence

if __name__ == '__main__':
    # Run the Flask app in debug mode
    app.run(debug=True)
